%%html
<div style="background:#f9f9f9; padding:15px; border-radius:10px; border:1px solid #ccc; font-family:Arial, sans-serif;">
  <h3 style="color:#2c3e50; text-align:center;">Shor's Algorithm – Step-by-Step Mathematics</h3>
  <p><b>Author:</b> PUJAN PANDEY</p>

  <p><b>1. The Problem:</b> Factor a composite integer <b>N</b>.</p>
  
  <p><b>2. Choose a random integer a:</b> $1 < a < N$, with $\gcd(a, N) = 1$.
     If $\gcd(a, N) \ne 1$, we already have a factor.
  </p>

  <p><b>3. Define function:</b><br>
     $$f(x) = a^x \mod N$$
     The period <b>r</b> is the smallest integer satisfying:
     $$a^r \equiv 1 \pmod{N}$$
  </p>

  <p><b>4. Quantum Phase Estimation:</b> Create superposition and apply modular exponentiation:
     $$|x\rangle \rightarrow |a^x \mod N\rangle$$
     Apply QFT to estimate the phase and find period $r$.
  </p>

  <p><b>5. Classical Post-Processing:</b> If r is even and $a^{r/2} \not\equiv -1 \mod N$, compute:
     $$p = \gcd(a^{r/2} - 1, N), \quad q = \gcd(a^{r/2} + 1, N)$$
     These are non-trivial factors of N.
  </p>

  <p><b>6. Example: N = 15</b><br>
     Pick a = 2 → r = 4 (since $2^4 \equiv 1 \mod 15$)<br>
     Factors: $\gcd(2^2 - 1, 15) = 3$, $\gcd(2^2 + 1, 15) = 5$
  </p>

  <p><b>7. Complexity:</b> Classical factoring: exponential; Shor's algorithm: $O((\log N)^3)$ (polynomial time)</p>

  <p style="text-align:right; font-size:12px; color:#555;">Made by PUJAN PANDEY</p>
</div>


In [9]:
import matplotlib.pyplot as plt
import qiskit
from qiskit import QuantumCircuit, generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import SparsePauliOp

from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, EstimatorV2 as Estimator

from qiskit_aer import AerSimulator

In [6]:
print(qiskit.__version__)

1.4.3


In [7]:
from qiskit.primitives import sampler

In [8]:
# Shor's Algorithm (from scratch) for small N using QPE + permutation unitary
# Works with Qiskit 1.4.3
import math
import random
import numpy as np

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Operator
from qiskit.primitives import Sampler

# ---------- Utilities ----------

def inv_qft(circ, qreg):
    """Append inverse QFT on the given register."""
    n = len(qreg)
    qft = QFT(num_qubits=n, inverse=True, do_swaps=True, name="QFT†")
    circ.append(qft.to_instruction(), qreg)

def continued_fraction(x, max_den=1 << 16):
    """Best rational approximation p/q to real x with q <= max_den."""
    # Standard continued fraction expansion
    a = []
    y = x
    for _ in range(32):  # plenty for our use
        ai = int(math.floor(y))
        a.append(ai)
        frac = y - ai
        if frac < 1e-12:
            break
        y = 1.0 / frac

    # Build convergents
    num1, num0 = 1, 0
    den1, den0 = 0, 1
    for ai in a:
        num = ai * num1 + num0
        den = ai * den1 + den0
        if den > max_den:
            break
        num0, num1 = num1, num
        den0, den1 = den1, den
    return num1, den1

def is_coprime(a, N):
    return math.gcd(a, N) == 1

def modular_order(a, N, r_guess):
    """Check if r_guess is the multiplicative order of a mod N."""
    if r_guess <= 0:
        return False
    return pow(a, r_guess, N) == 1

# ---------- Build the multiplication-by-a unitary on m qubits ----------

def mul_a_mod_N_unitary(a, N, m):
    """
    Returns a 2^m x 2^m permutation matrix U implementing:
      |y> -> |(a * y) mod N> for y < N
      |y> -> |y>            for y >= N   (padding for unitarity)
    """
    dim = 1 << m
    U = np.zeros((dim, dim), dtype=complex)
    for y in range(dim):
        if y < N:
            y_prime = (a * y) % N
        else:
            y_prime = y  # leave the invalid states untouched
        U[y_prime, y] = 1.0
    return U

def power_unitary(U, k):
    """Raise a unitary matrix U to power k (integer) by repeated squaring."""
    if k == 0:
        return np.eye(U.shape[0], dtype=complex)
    result = np.eye(U.shape[0], dtype=complex)
    base = U.copy()
    p = k
    while p > 0:
        if p & 1:
            result = result @ base
        base = base @ base
        p >>= 1
    return result

# ---------- QPE for order finding ----------

def qpe_order_finding(a, N, t=8):
    """
    Quantum Phase Estimation on the unitary U|y> = |a*y mod N>
    Returns an estimated phase s/r ~ phi, from which r is inferred.
    """
    # Target register size (enough to hold numbers up to N-1)
    m = int(math.ceil(math.log2(N)))
    dim = 1 << m

    # Build U_a
    U = mul_a_mod_N_unitary(a, N, m)

    # Registers
    count = QuantumRegister(t, "count")
    target = QuantumRegister(m, "target")
    c_out = ClassicalRegister(t, "c")
    qc = QuantumCircuit(count, target, c_out, name=f"QPE_order(a={a},N={N})")

    # Initialize |1> in target (a standard choice)
    qc.x(target[0])

    # Hadamards on counting register
    qc.h(count)

    # Controlled-U^(2^k) for k=0..t-1 (little-endian controls)
    for k in range(t):
        U_pow = power_unitary(U, 1 << k)
        U_gate = Operator(U_pow).to_instruction()
        CU = U_gate.control(1)
        qc.append(CU, [count[k]] + list(target))

    # Inverse QFT on counting register
    inv_qft(qc, count)

    # Measure counting register
    qc.measure(count, c_out)

    # Sample
    sampler = Sampler()
    job = sampler.run(qc, shots=4096)
    result = job.result()
    counts = result.quasi_dists[0]  # dict-like: bitstring->probability

    # Pick the most likely outcome
    # Keys are bitstrings like '01011010' with count[0] the LSB measured into c[0]
    best_bits, _ = max(counts.items(), key=lambda kv: kv[1])
    # Convert to integer (bitstring is big-endian by Qiskit convention for classical regs)
    s = int(best_bits, 2)
    frac = s / (1 << t)
    # Continued fraction to recover r
    p, q = continued_fraction(frac, max_den=N)
    r = q
    return r, s, frac, counts

# ---------- Full Shor wrapper ----------

def shor_factor(N, max_trials=8, t_count=8, rng_seed=None):
    """
    Factor a small composite N using Shor's algorithm from scratch.
    Returns (p, q, meta) where p*q==N (order not guaranteed).
    """
    if N % 2 == 0:
        return 2, N // 2, {"reason": "even"}
    # check perfect power of prime? (optional)

    rng = random.Random(rng_seed)

    for _ in range(max_trials):
        a = rng.randrange(2, N - 1)
        g = math.gcd(a, N)
        if g > 1:
            # lucky classical shortcut
            return g, N // g, {"a": a, "reason": "gcd>1 (classical)"}

        # Run QPE order finding
        r, s, frac, counts = qpe_order_finding(a, N, t=t_count)

        # If r odd, or a^(r/2) ≡ -1 (mod N), retry
        if r is None or r <= 0:
            continue
        if r % 2 == 1:
            continue
        if pow(a, r // 2, N) == N - 1:
            continue

        # Compute nontrivial factors
        x = pow(a, r // 2, N)
        p = math.gcd(x - 1, N)
        q = math.gcd(x + 1, N)
        if p in (1, N) or q in (1, N):
            continue
        if p * q == N:
            return p, q, {"a": a, "r": r, "s": s, "phase": frac}

    return None, None, {"reason": "failed_to_find_factors", "hint": "try increasing t_count or trials"}

# ---------- Example usage ----------
if __name__ == "__main__":
    N = 15          # try 15, 21, 33 ... (small composites)
    p, q, meta = shor_factor(N, max_trials=10, t_count=8, rng_seed=42)
    print("N =", N)
    print("Result:", p, q)
    print("Details:", meta)


N = 15
Result: 3 5
Details: {'a': 12, 'reason': 'gcd>1 (classical)'}
